In [1]:
import pandas as pd

In [ ]:
blast = pd.read_csv('/home/collot/stage/collot/collot/out_ortho/hs_blast/out/A1BG_9606.out', sep='\t', header=None)
blast.columns = ["query", "subject", "identity", "alignment_length", "mismatches", "gap_opens","q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]
mask = blast['query'] == blast['subject']
blast = blast[~mask] 
print(blast)

                           query                        subject  identity  \
1        A1BG_NM_130786.4-8_9606    A1BG_XM_030819447.1-7_61853    97.059   
2        A1BG_NM_130786.4-8_9606     A1BG_XM_001146669.7-8_9598    99.432   
3        A1BG_NM_130786.4-8_9606     A1BG_XM_055103953.3-8_9597    99.432   
4        A1BG_NM_130786.4-8_9606     A1BG_XM_031004187.3-8_9595    98.864   
5        A1BG_NM_130786.4-8_9606     A1BG_XM_055239924.2-8_9590    98.295   
...                          ...                            ...       ...   
4788  A1BG_XM_001146669.7-1_9598        A1BG_NM_130786.4-1_9606   100.000   
4789  A1BG_XM_001146669.7-1_9598     A1BG_XM_054464970.2-1_9600    95.556   
4790  A1BG_XM_001146669.7-1_9598     A1BG_XM_055239924.2-1_9590    95.556   
4791  A1BG_XM_001146669.7-1_9598  A1BG_XM_017509623.2-1_2715852    90.099   
4792  A1BG_XM_001146669.7-1_9598     A1BG_XM_031004187.3-1_9595    98.214   

      alignment_length  mismatches  gap_opens  q_start  q_end  s_start  s_e

In [ ]:
#get recirpocal best hits
best_hit=[]
for gene in blast['query'].unique() :
    df_gene = blast[blast['query']==gene]
    best_hit.append(df_gene.iloc[0])
best_df = pd.DataFrame(best_hit)


print(best_df)
reci=[]
for index, row in best_df.iterrows():
    que = row['query']
    sub = row['subject']
    if que in best_df['subject'].values:
        match = best_df[best_df['subject']==que]
        if sub in match['query'].values:
            reci.append(row)
reci_best_hit = pd.DataFrame(reci)
print(reci_best_hit)

                           query                      subject  identity  \
1        A1BG_NM_130786.4-8_9606  A1BG_XM_030819447.1-7_61853    97.059   
17       A1BG_NM_130786.4-7_9606   A1BG_XM_001146669.7-7_9598    99.306   
46       A1BG_NM_130786.4-6_9606  A1BG_XM_030819447.1-5_61853    97.872   
79       A1BG_NM_130786.4-5_9606   A1BG_XM_055103953.3-5_9597    98.316   
106      A1BG_NM_130786.4-4_9606   A1BG_XM_001146669.7-4_9598    98.168   
...                          ...                          ...       ...   
4699  A1BG_XM_001146669.7-5_9598   A1BG_XM_055103953.3-5_9597    99.663   
4727  A1BG_XM_001146669.7-4_9598   A1BG_XM_031004187.3-4_9595    98.535   
4754  A1BG_XM_001146669.7-3_9598   A1BG_XM_055103953.3-3_9597    99.259   
4783  A1BG_XM_001146669.7-2_9598   A1BG_XM_055103953.3-2_9597   100.000   
4788  A1BG_XM_001146669.7-1_9598      A1BG_NM_130786.4-1_9606   100.000   

      alignment_length  mismatches  gap_opens  q_start  q_end  s_start  s_end  \
1                 